<a href="https://colab.research.google.com/github/jaehyung-web/toilet/blob/main/%ED%99%94%EC%9E%A5%EC%8B%A4_%EC%BD%94%EB%93%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# 1. 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

# 2. 이미지 복사 (드라이브 > Colab Images 폴더에 campus.webp 올려놓기)
import os, shutil

# static 폴더 생성
os.makedirs("static", exist_ok=True)

# 복사 경로 지정
source_path = "/content/drive/MyDrive/static/campus.webp"
destination_path = "static/campus.webp"

# 이미지 복사
if os.path.exists(source_path):
    shutil.copy(source_path, destination_path)
    print("✅ 이미지 복사 완료: static/campus.webp")
else:
    print("❌ 이미지가 해당 경로에 없음. 먼저 'Colab Images' 폴더에 campus.webp 업로드해줘.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ 이미지 복사 완료: static/campus.webp


SyntaxError: invalid syntax (ipython-input-6-3091605603.py, line 1)

In [7]:
# app.py
# Flask + ngrok 서버 (데스크톱/모바일 반응형)
from flask import Flask, request, render_template_string
from pyngrok import ngrok

# ngrok 인증 토큰 설정
gnrok_token = "2zdNYkwML41M7L22D8Cd8wdW5xZ_74XsAp7x9DtveZ7xnHdgn"
ngrok.set_auth_token(gnrok_token)
app = Flask(__name__)

# 상태 저장 딕셔너리 (건물별 > 층별 > 위치 > 칸)
stall_status = {
    "본관": {"2": {"좌측": {str(i):0 for i in range(1,5)}, "우측": {str(i):0 for i in range(1,5)}},
            "3": {"좌측": {str(i):0 for i in range(1,5)}, "우측": {str(i):0 for i in range(1,5)}},
            "4": {"좌측": {str(i):0 for i in range(1,5)}, "우측": {str(i):0 for i in range(1,5)}},
            "5": {"좌측": {str(i):0 for i in range(1,5)}, "우측": {str(i):0 for i in range(1,5)}}},
    "아단관": {"4": {"7칸": {str(i):0 for i in range(1,8)}, "3칸": {str(i):0 for i in range(1,4)}},
              "5": {"7칸": {str(i):0 for i in range(1,8)}, "3칸": {str(i):0 for i in range(1,4)}}},
    "창의관": {str(f): {"화장실": {str(i):0 for i in range(1,5)}} for f in range(1,6)},
    "현암관": {"B1": {"화장실": {str(i):0 for i in range(1,6)}}, "1": {"화장실": {str(i):0 for i in range(1,6)}}},
    "여고": {str(f): {"화장실": {str(i):0 for i in range(1,5)}} for f in range(2,6)},
    "여송학사": {str(f): {"A동": {str(i):0 for i in range(1,5)}, "B동": {str(i):0 for i in range(1,5)}} for f in range(1,5)},
    "야구장": {}
}

# 센서 업데이트 엔드포인트
@app.route("/update")
def update():
    b = request.args.get("building")
    f = request.args.get("floor")
    loc = request.args.get("location")
    sid = request.args.get("stall")
    occ = request.args.get("occupied")
    if b in stall_status and f in stall_status[b] and loc in stall_status[b][f] and sid in stall_status[b][f][loc] and occ in ["0","1"]:
        stall_status[b][f][loc][sid] = int(occ)
        return "OK",200
    return "Bad Request",400

# 메인 페이지 (건물 선택)
main_page = '''
<!doctype html>
<html lang="ko">
<head><meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>화장실 건물 선택</title>
<style>
  body { margin:0; padding:0; font-family:Arial,sans-serif; text-align:center; }
  h1 { margin:20px 0; font-size:28px; }
  .grid {
    display:grid;
    grid-template-columns: repeat(auto-fit, minmax(160px,1fr));
    gap:16px;
    padding:16px;
  }
  button {
    width:100%; height:120px;
    font-size:22px;
    font-weight:bold;
    border:none;
    border-radius:12px;
    background:#3498db;
    color:#fff;
    cursor:pointer;
    transition:background 0.2s;
  }
  button:hover { background:#2980b9; }
</style>
</head>
<body>
<h1>BUGIL 실시간 화장실 정보 시스템</h1>
<h2>화장실을 선택하십시오.</h2>
<div class="grid">
{% for b in buildings %}
  <form action="/building/{{ b }}">
    <button type="submit">{{ b }}</button>
  </form>
{% endfor %}
</div>
</body>
</html>
'''

# 건물별 현황 페이지
building_page = '''
<!doctype html>
<html lang="ko">
<head><meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>{{ building }} 화장실 현황</title>
<style>
  body { margin:0; padding:16px; font-family:Arial,sans-serif; }
  h1 { font-size:24px; margin-bottom:12px; }
  h2 { font-size:20px; margin:16px 0 8px; }
  h3 { font-size:18px; margin:12px 0 6px; }
  .container {
    display:grid;
    grid-template-columns: repeat(auto-fit, minmax(100px,1fr));
    gap:8px;
    margin-bottom:12px;
  }
  .box {
    padding:16px;
    text-align:center;
    border-radius:8px;
    color:#fff;
    font-weight:bold;
    font-size:16px;
  }
  .occupied { background:#e74c3c; }
  .vacant { background:#2ecc71; }
  a.back {
    display:inline-block;
    margin-top:16px;
    padding:12px 24px;
    font-size:18px;
    background:#555;
    color:#fff;
    border-radius:8px;
    text-decoration:none;
  }
  a.back:hover { background:#333; }
</style>
</head>
<body>
<h1>{{ building }} 화장실 현황</h1>
{% for floor, places in data.items() %}
  <h2>{{ floor }}층</h2>
  {% for loc, stalls in places.items() %}
    <h3>{{ loc }}</h3>
    <div class="container">
      {% for sid, occ in stalls.items() %}
        <div class="box {{ 'occupied' if occ else 'vacant' }}">
          {{ sid }}번<br>{{ '사용중' if occ else '비어있음' }}
        </div>
      {% endfor %}
    </div>
  {% endfor %}
{% endfor %}
<a href="/" class="back"> 돌아가기</a>
</body>
</html>
'''

@app.route("/")
def home():
    return render_template_string(main_page, buildings=stall_status.keys())

@app.route("/building/<b>")
def show_building(b):
    if b in stall_status:
        return render_template_string(building_page, building=b, data=stall_status[b])
    return "Not Found",404

# ngrok 공개 URL
public_url = ngrok.connect(5000)
print("✅ URL:", public_url)

if __name__ == "__main__":
    app.run(port=5000)


ModuleNotFoundError: No module named 'pyngrok'

In [10]:
# app.py
#최종코드
# Flask + ngrok 서버 (데스크톱/모바일 반응형)
from flask import Flask, request, render_template_string
from pyngrok import ngrok

# ngrok 인증 토큰 설정
gnrok_token = "2zdNYkwML41M7L22D8Cd8wdW5xZ_74XsAp7x9DtveZ7xnHdgn"
ngrok.set_auth_token(gnrok_token)
app = Flask(__name__)

# 상태 저장 딕셔너리 (건물별 > 층별 > 위치 > 칸)
stall_status = {
    "본관": {"2": {"좌측": {str(i):0 for i in range(1,5)}, "우측": {str(i):0 for i in range(1,5)}},
            "3": {"좌측": {str(i):0 for i in range(1,5)}, "우측": {str(i):0 for i in range(1,5)}},
            "4": {"좌측": {str(i):0 for i in range(1,5)}, "우측": {str(i):0 for i in range(1,5)}},
            "5": {"좌측": {str(i):0 for i in range(1,5)}, "우측": {str(i):0 for i in range(1,5)}}},
    "아단관": {"4": {"7칸": {str(i):0 for i in range(1,8)}, "3칸": {str(i):0 for i in range(1,4)}},
              "5": {"7칸": {str(i):0 for i in range(1,8)}, "3칸": {str(i):0 for i in range(1,4)}}},
    "창의관": {str(f): {"화장실": {str(i):0 for i in range(1,5)}} for f in range(1,6)},
    "현암관": {"B1": {"화장실": {str(i):0 for i in range(1,6)}}, "1": {"화장실": {str(i):0 for i in range(1,6)}}},
    "여고": {str(f): {"화장실": {str(i):0 for i in range(1,5)}} for f in range(2,6)},
    "여송학사": {str(f): {"A동": {str(i):0 for i in range(1,5)}, "B동": {str(i):0 for i in range(1,5)}} for f in range(1,5)},
    "야구장": {}
}

# 센서 업데이트 엔드포인트
@app.route("/update")
def update():
    b = request.args.get("building")
    f = request.args.get("floor")
    loc = request.args.get("location")
    sid = request.args.get("stall")
    occ = request.args.get("occupied")
    if b in stall_status and f in stall_status[b] and loc in stall_status[b][f] and sid in stall_status[b][f][loc] and occ in ["0","1"]:
        stall_status[b][f][loc][sid] = int(occ)
        return "OK",200
    return "Bad Request",400

# 메인 페이지 (건물 선택)
main_page = '''
<!doctype html>
<html lang="ko">
<head><meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>화장실 건물 선택</title>
<style>
  body { margin:0; padding:0; font-family:Arial,sans-serif; text-align:center; }
  h1 { margin:20px 0; font-size:28px; }
  h2 { margin:10px 0 30px; font-size:24px; font-weight:normal; color:#555; }
  .grid {
    display:grid;
    grid-template-columns: repeat(auto-fit, minmax(160px,1fr));
    gap:16px;
    padding:16px;
  }
  button {
    width:100%; height:120px;
    font-size:22px;
    font-weight:bold;
    border:none;
    border-radius:12px;
    background:#f39733;
    color:#fff;
    cursor:pointer;
    transition:background 0.2s;
  }
  button:hover { background:#d37f2e; }
</style>
</head>
<body>
<h1>BUGIL 실시간 화장실 정보 시스템</h1>
<h2>화장실을 선택하십시오.</h2>
<div class="grid">
{% for b in buildings %}
  <form action="/building/{{ b }}">
    <button type="submit">{{ b }}</button>
  </form>
{% endfor %}
</div>
</body>
</html>
'''

# 건물별 현황 페이지
building_page = '''
<!doctype html>
<html lang="ko">
<head><meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0">
<title>{{ building }} 화장실 현황</title>
<style>
  body { margin:0; padding:16px; font-family:Arial,sans-serif; }
  h1 { font-size:22px; margin-bottom:12px; }
  h2 { font-size:20px; margin:16px 0 8px; }
  h3 { font-size:18px; margin:12px 0 6px; }
  .container {
    display:grid;
    grid-template-columns: repeat(auto-fit, minmax(100px,1fr));
    gap:8px;
    margin-bottom:12px;
  }
  .box {
    padding:16px;
    text-align:center;
    border-radius:8px;
    color:#fff;
    font-weight:bold;
    font-size:16px;
  }
  .occupied { background:#e74c3c; }
  .vacant { background:#2ecc71; }
  a.back {
    display:inline-block;
    margin-top:16px;
    padding:12px 24px;
    font-size:18px;
    background:#555;
    color:#fff;
    border-radius:8px;
    text-decoration:none;
  }
  a.back:hover { background:#333; }
</style>
</head>
<body>
<h1>{{ building }} 화장실 현황</h1>
{% for floor, places in data.items() %}
  <h2>{{ floor }}층</h2>
  {% for loc, stalls in places.items() %}
    <h3>{{ loc }}</h3>
    <div class="container">
      {% for sid, occ in stalls.items() %}
        <div class="box {{ 'occupied' if occ else 'vacant' }}">
          {{ sid }}번<br>{{ '사용중' if occ else '비어있음' }}
        </div>
      {% endfor %}
    </div>
  {% endfor %}
{% endfor %}
<a href="/" class="back">⬅️ 돌아가기</a>
</body>
</html>
'''

@app.route("/")
def home():
    return render_template_string(main_page, buildings=stall_status.keys())

@app.route("/building/<b>")
def show_building(b):
    if b in stall_status:
        return render_template_string(building_page, building=b, data=stall_status[b])
    return "Not Found",404

# ngrok 공개 URL
public_url = ngrok.connect(5000)
print("✅ URL:", public_url)

if __name__ == "__main__":
    app.run(port=5000)


✅ URL: NgrokTunnel: "https://71438903a86c.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [4]:
# app.py
# Flask 서버 + Static 이미지 지원 (Colab 구글 드라이브 마운트 포함)
from flask import Flask, request, render_template_string, send_from_directory
import os, shutil
# Colab 드라이브 마운트 시 자동 처리
try:
    from google.colab import drive
    drive.mount('/content/drive')
except ImportError:
    pass
# ngrok (선택 사용)
try:
    from pyngrok import ngrok
    use_ngrok = True
    ngrok.set_auth_token("2zdNYkwML41M7L22D8Cd8wdW5xZ_74XsAp7x9DtveZ7xnHdgn")  # ngrok 토큰
except ImportError:
    use_ngrok = False

# 사용자 설정: 구글 드라이브 내 이미지 경로 (Colab 전용)
LOGO_DRIVE_PATH = "/content/drive/MyDrive/static/images (5).png"  # 여기에 붙여넣으세요

# static 폴더 준비 및 이미지 복사
if os.path.exists(LOGO_DRIVE_PATH):
    os.makedirs('static', exist_ok=True)
    shutil.copy(LOGO_DRIVE_PATH, 'static/logo.png')
else:
    # 기본 로고 제거 시 대비
    os.makedirs('static', exist_ok=True)
    if not os.path.exists('static/logo.png'):
        open('static/logo.png','a').close()

app = Flask(__name__)

# 화장실 상태 데이터
stall_status = {
    "본관": {"2":{"좌측":{str(i):0 for i in range(1,5)},"우측":{str(i):0 for i in range(1,5)}},
             "3":{"좌측":{str(i):0 for i in range(1,5)},"우측":{str(i):0 for i in range(1,5)}},
             "4":{"좌측":{str(i):0 for i in range(1,5)},"우측":{str(i):0 for i in range(1,5)}},
             "5":{"좌측":{str(i):0 for i in range(1,5)},"우측":{str(i):0 for i in range(1,5)}}},
    "아단관": {"4":{"7칸":{str(i):0 for i in range(1,8)},"3칸":{str(i):0 for i in range(1,4)}},
              "5":{"7칸":{str(i):0 for i in range(1,8)},"3칸":{str(i):0 for i in range(1,4)}}},
    "창의관": {str(f):{"화장실":{str(i):0 for i in range(1,5)}} for f in range(1,6)},
    "현암관":{"B1":{"화장실":{str(i):0 for i in range(1,6)}},"1":{"화장실":{str(i):0 for i in range(1,6)}}},
    "여고":{str(f):{"화장실":{str(i):0 for i in range(1,5)}} for f in range(2,6)},
    "여송학사":{str(f):{"A동":{str(i):0 for i in range(1,5)},"B동":{str(i):0 for i in range(1,5)}} for f in range(1,5)},
    "야구장":{}
}

# 센서 업데이트 라우트
@app.route('/update')
def update():
    b = request.args.get('building')
    f = request.args.get('floor')
    loc = request.args.get('location')
    sid = request.args.get('stall')
    occ = request.args.get('occupied')
    if all([b in stall_status,
            f in stall_status[b],
            loc in stall_status[b][f],
            sid in stall_status[b][f][loc],
            occ in ['0','1']]):
        stall_status[b][f][loc][sid] = int(occ)
        return 'OK',200
    return 'Bad Request',400

# static 파일 서빙
@app.route('/static/<path:filename>')
def static_files(filename):
    return send_from_directory('static', filename)

# 메인 페이지 템플릿
main_page = '''
<!doctype html>
<html lang='ko'>
<head><meta charset='utf-8'>
<meta name='viewport' content='width=device-width,initial-scale=1.0'>
<title>건물 선택</title>
<style>
  body{margin:0;padding:0;font-family:Arial,sans-serif;text-align:center;}
  h1{margin:20px 0;font-size:28px;}
  h2{margin:10px 0 30px;font-size:24px;color:#555;}
  .grid{display:grid;grid-template-columns:repeat(auto-fit,minmax(160px,1fr));gap:16px;padding:16px;}
  button{width:100%;height:120px;font-size:22px;font-weight:bold;border:none;border-radius:12px;background:#f39733;color:#fff;cursor:pointer;transition:0.2s;}
  button:hover{background:#d37f2e;}
  img.logo{width:100%;height:100%;object-fit:contain;}
</style>
</head>
<body>
<h1>BUGIL 실시간 화장실 정보 시스템</h1>
<h2>화장실을 선택하십시오.</h2>
<div class='grid'>
  {% for b in buildings %}
  <form action='/building/{{ b }}'><button>{{ b }}</button></form>
  {% endfor %}
  <!-- 이미지 버튼 -->
  <form action='#'><button type='button'><img src='/static/logo.png' class='logo'></button></form>
</div>
</body>
</html>
'''

# 건물 페이지 템플릿
building_page = '''
<!doctype html>
<html lang='ko'>
<head><meta charset='utf-8'>
<meta name='viewport' content='width=device-width,initial-scale=1.0'>
<title>{{ building }} 현황</title>
<style>
  body{margin:0;padding:16px;font-family:Arial,sans-serif;}
  h1{font-size:24px;margin-bottom:12px;}
  h2{font-size:20px;margin:16px 0 8px;}
  h3{font-size:18px;margin:12px 0 6px;}
  .container{display:grid;grid-template-columns:repeat(auto-fit,minmax(100px,1fr));gap:8px;margin-bottom:12px;}
  .box{padding:16px;text-align:center;border-radius:8px;color:#fff;font-weight:bold;font-size:16px;}
  .occupied{background:#e74c3c;}
  .vacant{background:#2ecc71;}
  a.back{display:inline-block;margin-top:16px;padding:12px 24px;font-size:18px;background:#555;color:#fff;border-radius:8px;text-decoration:none;}
  a.back:hover{background:#333;}
</style>
</head>
<body>
<h1>{{ building }} 화장실 현황</h1>
{% for floor, places in data.items() %}
  <h2>{{ floor }}층</h2>
  {% for loc, stalls in places.items() %}
    <h3>{{ loc }}</h3>
    <div class='container'>
      {% for sid, occ in stalls.items() %}
        <div class='box {{ 'occupied' if occ else 'vacant' }}'>{{ sid }}번<br>{{ '사용중' if occ else '비어있음' }}</div>
      {% endfor %}
    </div>
  {% endfor %}
{% endfor %}
<a href='/' class='back'>⬅️ 돌아가기</a>
</body>
</html>
'''

@app.route('/')
def home():
    return render_template_string(main_page, buildings=stall_status.keys())

@app.route('/building/<b>')
def show_building(b):
    if b in stall_status:
        return render_template_string(building_page, building=b, data=stall_status[b])
    return 'Not Found',404

# ngrok 연결
if use_ngrok:
    url = ngrok.connect(5000)
    print('✅ ngrok URL:', url)

if __name__=='__main__':
    app.run(port=5000)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [21]:
# app.py
# Flask 서버 + Static 이미지 지원 (Colab 구글 드라이브 마운트 포함)
import subprocess, sys
# 런타임에 반드시 필요한 패키지 설치
subprocess.check_call([sys.executable, "-m", "pip", "install", "flask", "pyngrok"] , stdout=subprocess.DEVNULL)

from flask import Flask, request, render_template_string, send_from_directory
import os, shutil

# Colab 환경에서 구글 드라이브 자동 마운트 (이미 마운트된 경우 건너뜀)
try:
    from google.colab import drive
    if not os.path.exists('/content/drive/MyDrive'):
        drive.mount('/content/drive')
except ImportError:
    pass

# ngrok 필수 사용
from pyngrok import ngrok
ngrok.set_auth_token("2zdNYkwML41M7L22D8Cd8wdW5xZ_74XsAp7x9DtveZ7xnHdgn")

app = Flask(__name__)

# 사용자 설정: 구글 드라이브 내 이미지 경로 (Colab 전용)
LOGO_DRIVE_PATH = "/content/drive/MyDrive/static/images (5).png"  # 실제 경로로 수정하세요

# static 폴더 준비 및 이미지 복사
os.makedirs('static', exist_ok=True)
if os.path.exists(LOGO_DRIVE_PATH):
    shutil.copy(LOGO_DRIVE_PATH, 'static/logo.png')
elif not os.path.exists('static/logo.png'):
    open('static/logo.png','a').close()

# 화장실 상태 데이터
stall_status = {
    "본관": {"2":{"좌측":{str(i):0 for i in range(1,5)},"우측":{str(i):0 for i in range(1,5)}},
             "3":{"좌측":{str(i):0 for i in range(1,5)},"우측":{str(i):0 for i in range(1,5)}},
             "4":{"좌측":{str(i):0 for i in range(1,5)},"우측":{str(i):0 for i in range(1,5)}},
             "5":{"좌측":{str(i):0 for i in range(1,5)},"우측":{str(i):0 for i in range(1,5)}}},
    "아단관": {"4":{"큰 화장실":{str(i):0 for i in range(1,8)},"작은 화장실":{str(i):0 for i in range(1,4)}},
              "5":{"큰 화장실":{str(i):0 for i in range(1,8)},"작은 화장실":{str(i):0 for i in range(1,4)}}},
    "창의관": {str(f):{"화장실":{str(i):0 for i in range(1,5)}} for f in range(1,6)},
    "현암관":{"B1":{"화장실":{str(i):0 for i in range(1,6)}},"1":{"화장실":{str(i):0 for i in range(1,6)}}},
    "여고":{str(f):{"화장실":{str(i):0 for i in range(1,5)}} for f in range(2,6)},
    "여송학사":{str(f):{"A동":{str(i):0 for i in range(1,5)},"B동":{str(i):0 for i in range(1,5)}} for f in range(1,5)},
    "야구장":{}
}

# 센서 업데이트 라우트
@app.route('/update')
def update():
    b = request.args.get('building')
    f = request.args.get('floor')
    loc = request.args.get('location')
    sid = request.args.get('stall')
    occ = request.args.get('occupied')
    if all([b in stall_status,
            f in stall_status[b],
            loc in stall_status[b][f],
            sid in stall_status[b][f][loc],
            occ in ['0','1']]):
        stall_status[b][f][loc][sid] = int(occ)
        return 'OK',200
    return 'Bad Request',400

# static 파일 서빙
@app.route('/static/<path:filename>')
def static_files(filename):
    return send_from_directory('static', filename)

# 메인 페이지 템플릿
main_page = '''
<!doctype html>
<html lang='ko'>
<head><meta charset='utf-8'>
<meta name='viewport' content='width=device-width,initial-scale=1.0'>
<title>BUGIL 실시간 화장실 정보 시스템</title>
<style>
  body{margin:0;padding:0;font-family:Arial,sans-serif;text-align:center;}
  h1{margin:20px 0;font-size:28px;}
  h2{margin:10px 0 30px;font-size:24px;color:#555;}
  .grid{display:grid;grid-template-columns:repeat(auto-fit,minmax(160px,1fr));gap:16px;padding:16px;}
  button{width:100%;height:120px;font-size:22px;font-weight:bold;border:none;border-radius:12px;background:#f39733;color:#fff;cursor:pointer;transition:0.2s;}
  button:hover{background:#d37f2e;}
  /* 로고 버튼만 배경 제거 */
  button.logo-btn { background: none; }
  button.logo-btn:hover { background: none; }
  img.logo{width:100%;height:100%;object-fit:contain;}
</style>
</head>
<body>
<h1>BUGIL 실시간 화장실 정보 시스템</h1>
<h2>화장실을 선택하십시오.</h2>
<div class='grid'>
  {% for b in buildings %}
  <form action='/building/{{ b }}'><button>{{ b }}</button></form>
  {% endfor %}
  <!-- 이미지 버튼 -->
  <form action='#'><button type='button' class='logo-btn'><img src='/static/logo.png' class='logo'></button></form>
</div>
</body>
</html>
'''

# 건물 페이지 템플릿
building_page = '''
<!doctype html>
<html lang='ko'>
<head><meta charset='utf-8'>
<meta name='viewport' content='width=device-width,initial-scale=1.0'>
<title>{{ building }} 현황</title>
<style>
  body{margin:0;padding:16px;font-family:Arial,sans-serif;}
  h1{font-size:24px;margin-bottom:12px;}
  h2{font-size:20px;margin:16px 0 8px;}
  h3{font-size:18px;margin:12px 0 6px;}
  .container{display:grid;grid-template-columns:repeat(auto-fit,minmax(100px,1fr));gap:8px;margin-bottom:12px;}
  .box{padding:16px;text-align:center;border-radius:8px;color:#fff;font-weight:bold;font-size:16px;}
  .occupied{background:#e74c3c;}
  .vacant{background:#2ecc71;}
  a.back{display:inline-block;margin-top:16px;padding:12px 24px;font-size:18px;background:#555;color:#fff;border-radius:8px;text-decoration:none;}
  a.back:hover{background:#333;}
</style>
</head>
<body>
<h1>{{ building }} 화장실 현황</h1>
{% for floor, places in data.items() %}
  <h2>{{ floor }}층</h2>
  {% for loc, stalls in places.items() %}
    <h3>{{ loc }}</h3>
    <div class='container'>
      {% for sid, occ in stalls.items() %}
        <div class='box {{ 'occupied' if occ else 'vacant' }}'>{{ sid }}번<br>{{ '사용중' if occ else '비어있음' }}</div>
      {% endfor %}
    </div>
  {% endfor %}
{% endfor %}
<a href='/' class='back'>⬅ 돌아가기</a>
</body>
</html>
'''

@app.route('/')
def home():
    return render_template_string(main_page, buildings=stall_status.keys())

@app.route('/building/<b>')
def show_building(b):
    if b in stall_status:
        return render_template_string(building_page, building=b, data=stall_status[b])
    return 'Not Found',404

# ngrok 터널링
url = ngrok.connect(5000)
print('✅ ngrok URL:', url)

if __name__=='__main__':
    app.run(port=5000)

✅ ngrok URL: NgrokTunnel: "https://d4aba23333ba.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [11/Jul/2025 01:35:05] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jul/2025 01:35:06] "GET /static/logo.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jul/2025 01:35:06] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Jul/2025 01:36:41] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jul/2025 01:36:42] "GET /static/logo.png HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jul/2025 01:36:49] "GET /building/본관 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jul/2025 01:37:05] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jul/2025 01:37:06] "GET /static/logo.png HTTP/1.1" 304 -
INFO:werkzeug:127.0.0.1 - - [11/Jul/2025 01:37:08] "GET /building/아단관 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jul/2025 01:37:11] 